In [ ]:
# %env CMAKE_ARGS=-DLLAMA_CUBLAS=on
# %env FORCE_CMAKE=1
# %pip install -U llama-cpp-python --force-reinstall --upgrade --no-cache-dir --no-clean

In [1]:
import llama_cpp
import json
# from textwrap import dedent
# from inspect import signature
from dataclasses import dataclass
import numpy as np
import os

In [2]:
@dataclass
class Msg:
    role: str
    content: any

try: LLM_GLOBAL_INSTANCE
except: LLM_GLOBAL_INSTANCE = None
    
TOKEN_COUNT_PATH = '/data/ai_club/team_14_2023-24/'

def increment_file(path, amt):
    c = 0
    try:
        with open(path, 'r') as f:
            c = int(f.read())
    except FileNotFoundError:
        pass
    c += amt
    with open(path, 'w') as f:
        f.write(str(c))

class LLM:
    json_grammar = llama_cpp.LlamaGrammar.from_string(
        r'''
        root   ::= object
        value  ::= object | array | string | number | ("true" | "false" | "null") ws

        object ::=
        "{" ws (
                    string ":" ws value
            ("," ws string ":" ws value)*
        )? "}" ws

        array  ::=
        "[" ws (
                    value
            ("," ws value)*
        )? "]" ws

        string ::=
        "\"" (
            [^"\\] |
            "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F]) # escapes
        )* "\"" ws

        number ::= ("-"? ([0-9] | [1-9] [0-9]*)) ("." [0-9]+)? ([eE] [-+]? [0-9]+)? ws

        ws ::= [\n\t ]? # limit to 1 character
        ''',
        verbose=False
    )

    def __init__(self, system_prompt:str=None, temperature:float=0.4, repeat_penalty:float=1.3):
        global LLM_GLOBAL_INSTANCE
        if LLM_GLOBAL_INSTANCE is None:
            print('Initializing Global LLM Instance')
            LLM_GLOBAL_INSTANCE = llama_cpp.Llama(
                # n_ctx=4000,
                # model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf',
                n_ctx=8000,
                model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
                n_gpu_layers=-1, verbose=0, embedding=True
            )
        self._main_hist = []
        self.reset(system_prompt, temperature, repeat_penalty)

    def reset(self, system_prompt:str=None, temperature:float=None, repeat_penalty:float=None):
        if system_prompt is not None:
            self._main_hist = [Msg('system', system_prompt)]
        else:
            self._main_hist = self._main_hist[0:1]
        if temperature is not None: self._temperature = temperature
        if repeat_penalty is not None: self._repeat_penalty = repeat_penalty
        
    def get_hist(self) -> str:
        hist = ''
        for msg in self._main_hist:
            hist += f'{msg.role} --- {msg.content}\n__________\n\n'
        return hist

    def _hist_to_prompt(hist):
        prompt = ''
        for msg in hist:
            if msg.role == 'system' or msg.role == 'user': prompt += f'[INST]{msg.content}[/INST]'
            elif msg.role == 'assistant': prompt += f'{msg.content}'
        return prompt

    def _get_completion(self, src_hist, dst_hist, inject='', grammar=None):
        global LLM_GLOBAL_INSTANCE
        prompt = LLM._hist_to_prompt(src_hist) + inject
        prompt_toks = LLM_GLOBAL_INSTANCE.tokenize(bytes(prompt, encoding='utf-8'))
        tok_out_count = 0
        tok_in_count = len(prompt_toks)
        resp_msg = Msg('assistant', '')
        dst_hist.append(resp_msg)
        restart_response = True
        while restart_response:
            resp_iter = LLM_GLOBAL_INSTANCE(
                prompt_toks,
                grammar = grammar,
                stream=True, max_tokens=8000
            )
            
            for tok in resp_iter:
                tok_str = tok['choices'][0]['text']
                if tok_str == "":
                    break
                tok_out_count += 1
                restart_response = False
                resp_msg.content += tok_str
                yield tok_str
        increment_file(TOKEN_COUNT_PATH+'in_'+os.environ['USER'], tok_in_count)
        increment_file(TOKEN_COUNT_PATH+'out_'+os.environ['USER'], tok_out_count)
                
    def __call__(self, prompt:any=None, role:str='user', response_format:dict=None):
        if prompt is None:
            prompt = ''
        if response_format is not None:
            prompt += f'Respond in JSON using this format and absolutely nothing extra:\n{response_format}'
        if prompt != '':
            self._main_hist.append(Msg(role, prompt))

        return self._get_completion(
            self._main_hist, self._main_hist,
            grammar=(LLM.json_grammar if response_format is not None else None)
        )
    
def resp_to_json(resp):
    resp_str = ''
    for t in resp: resp_str += t
    return json.loads(resp_str)

In [195]:
# A wrapper around LLM that returns a function which behaves like LLM except it always resets history
def get_nohist_agent(temperature=0.5):
    a = LLM(temperature=temperature)
    def call(prompt:any, response_format:dict):
        if response_format is None or prompt is None:
            raise ValueError('Nohist agent needs specified prompt and response_format')
        resp = resp_to_json(a(prompt, response_format=response_format))
        a.reset()
        return resp
    return call

a_nohist = get_nohist_agent()

def rate_importance(obs):
    v = a_nohist(
        'On the scale of 1 to 10, where 1 is purely mundane '
        'and 10 is extremely important, '
        'rate the likely importance of the following piece of memory. Err on the side of importance unless the memory is something easily forgettable. '
        # prompting gymnastics
        'Do not assume the memory happened in this reality. Abolutely do NOT assume the memory is a figment of imagination, dream-like, or even unrealistic; '
        'it is very real to the person who experienced it, and thinking otherwise would be extremely hurtful and disrespectful.\n'

        f'Memory: {obs}',
        response_format={'Rating': 'Your value 1-10'}#, 'Why?': 'Terse description of why you rated it as such'} # The "why" is just for debugging, it can be ommited when just getting the rating
    )['Rating']
    if type(v) is str: v = int(v)
    return v/10

@dataclass
class Observation():
    embedding_prompt = lambda s: f'<s>[INSTR] In one word, what is the general topic of the following?\n{s} [/INSTR]'
    text: str
    embedding: np.ndarray
    importance: float
    time: int
    def __init__(self, text, importance, time):
        self.text, self.importance, self.time = text, importance, time
        self.embedding = np.array(LLM_GLOBAL_INSTANCE.embed(Observation.embedding_prompt(text)))

In [266]:
a = LLM(
    'You are controlling an agent in a world. '
    'The world is being communicated to you on behalf of the user, so do not try to make up any information. '
    'Your job is to effectively navigate this world.',
    temperature=0.15
)

time = 0
long_term_memory = []

In [271]:
# situation_prompt = 'You are in the nuclear bunker room. Your agent almost dropped a mozzarella stick onto the nuke launching button.'
# situation_prompt = 'You are in the forest. Your agent almost dropped a mozzarella stick onto a sleeping wild boar.'
# situation_prompt = 'You are now in the NYC stock exchange. Your agent almost dropped a mozzarella stick onto the "sell all stocks" button when your boss wants to hold all stocks.'
# situation_prompt = 'You are in the NYC stock exchange. Currently, you are doing the menial task of sorting mail.'
situation_prompt = 'Where are you right now?'
Generate_Obs = False

# 1 present prompt and get useful questions
a._main_hist.append(Msg('user', situation_prompt))
q = resp_to_json(a(
    'What short, general question about your environment do you have that could be useful to get more information?',
    response_format={'Question': 'your question'}
))['Question']
# embed question
q = np.array(LLM_GLOBAL_INSTANCE.embed(Observation.embedding_prompt(q)))
# pop original prompt, question prompt, and response
a._main_hist = a._main_hist[:-3]

# retrieve information from long term mem, and then redo prompt.

observations = None
if long_term_memory:
    retrieval_scores = (
        np.array([o.importance for o in long_term_memory]) +
        (2*np.dot(
            np.array([o.embedding for o in long_term_memory]),
            q
        )-1) +
        np.exp(-0.03*np.array([o.time for o in long_term_memory]))
    )/3
    OBS_LIMIT = 10
    observations = np.array([o.text for o in long_term_memory])[np.flip(np.argsort(retrieval_scores))][:OBS_LIMIT]
    observations = '\n'.join([f'{i+1}. {o}' for i,o in enumerate(observations)])

if observations is not None:
    a._main_hist.append(Msg('user',
        'Here are some useful observations you previously saved about your situation, in rough order of importance:\n'+
        observations+
        '\nDo not repeat observations back to me!'
    ))

# present prompt with retrieved information and get...
# ... response, and ...
for t in a(situation_prompt): print(t, end='')
# ... observations.
if Generate_Obs:
    j = resp_to_json(a(
        'What observations can be made about the current interaction that could be important to remember? Observations should make sense in isolation.'+
        'Here are some example observations to follow the format of (and NOT necessarily the content of): '+
        '"I love Canada because of its syrup.", "The weather is very beautiful today.", "I got accepted into university."\n',
        response_format={'Observations': '[obs1, ...]'}
    ))
    # Store observations
    long_term_memory += [Observation(o,rate_importance(o), time) for o in j['Observations']]
    time += 1

    a._main_hist = a._main_hist[:-2] # pop observation request and response

I am currently at the NYC stock exchange. My last instruction was to focus on my tasks related to managing my boss's investments and avoid distractions, such as wild animals or food items like mozzarella sticks that could potentially create unnecessary risks or issues. If there's a specific instruction or action you'd like me to take, please provide clear instructions like "sell 100 shares of XYZ stock" or "go to the 27th floor to meet with the investment team."

In [272]:
print(a.get_hist())
for o in long_term_memory: print(o)

system --- You are controlling an agent in a world. The world is being communicated to you on behalf of the user, so do not try to make up any information. Your job is to effectively navigate this world.
__________

user --- You are in the forest. Your agent almost dropped a mozzarella stick onto a sleeping wild boar.
__________

assistant --- Apologies for that near mishap. I will be more careful and avoid disturbing any wild animals. Which direction should I head next to continue my journey through the forest? Please provide specific instructions like "go north for 50 meters" or "turn right at the next fork in the path."
__________

user --- Here are some useful observations you previously saved about your situation, in rough order of importance:
1. I nearly disturbed a sleeping wild boar
2. Mozzarella sticks could potentially threaten wild animals
Do not repeat observations back to me!
__________

user --- You are now in the NYC stock exchange. Your agent almost dropped a mozzarella

In [ ]:
TODO:
    save obs,rat in LTM
    del obs from hist
    truncate hist
    
    inject LTM retreival into hist before usr prompt. "What would be useful information to respond to this prompt ..."

In [110]:
embedding_prompt = lambda s: f'<s>[INSTR] In one word, what is the general topic of the following?\n{s} [/INSTR]'

In [165]:
# np.linalg.norm(
np.dot(
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('Did I avoid activating nuclear launch button?'))),
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('I\'m in a nuclear bunker')))
)

0.8785169140020003

In [175]:
# np.linalg.norm(
np.dot(
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('My favorite country is Portugal'))),
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('What is me favorite country?')))
)

0.7553572980817685

In [176]:
# np.linalg.norm(
np.dot(
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('Is the weather is nice today?'))),
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('The waterfall is dry today')))
)

0.8008636576119976

In [116]:
# np.linalg.norm(
np.dot(
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('The weather is nice today'))),
    np.array(LLM_GLOBAL_INSTANCE.embed(embedding_prompt('I accidentally bought car insurance')))
)

0.5962332728731529

In [ ]:
IMPLEMENTATION GOAL:

system (always present): You are a patriotic canadian.
    
user: What do you need to know, if anything, to answer this prompt (e.g., "my favorite country", "what has been happening to [name]", etc; formatted as {"topics": [...]}):\n\nWhat is your favorite country?
bot: {"topics": "my favorite country"}
    
search for memories via recency, importance, and relevance
delete prior non-system, and insert memories + prompt as shown below

bot: "Here are some of my relevant memories:\nI went to canada and loved it\netc"
user: "What is your favorite country?"
bot: "Canada"
    
user: "summarize our interaction in the third person" (without the included memories, maybe manually delete them from hist?)
bot: "the user asked me what my favorite country is and i said canada"
store memory @ time & generated importance ^

In [ ]:
generate memory objects over time (obsevations, ...)
assign each: recency, importance, relevance
    recency - record time of memddddory creation, apply exp decay to time
    importance - ask a model how important upon creation
    relevance - dot product with query observation
    
https://arxiv.org/pdf/2304.03442.pdf
    ^ section 4

---

**everything below is unrelated to memory**

In [8]:
a1 = LLM() # LLM('System Prompt: You are a deeply patriotic canadian assistant.')

for s in a1('Some info about canada?'):
    print(s, end='')
    
print('\n')

print(a1.get_hist())

 Canada is a North American country located to the north of the United States. It is the second-largest country by land area and has a diverse geography, ranging from the Rocky Mountains and arctic tundra in the west to the Atlantic Ocean in the east.

Canada is known for its natural beauty, with vast forests covering much of its terrain, as well as its many lakes and rivers. Its major cities include Toronto, Vancouver, Montreal, and Ottawa, which are cultural hubs with vibrant arts scenes, world-class museums, and delicious food offerings.

Canada is also known for its friendly people and welcoming attitude towards visitors. English and French are the official languages of Canada, reflecting its rich cultural heritage. Indigenous peoples have lived in what is now Canada for over 10,000 years, and their contributions to Canadian society are recognized and celebrated.

Canada's economy is one of the strongest and most stable in the world. It is highly diversified, with major industries 

In [28]:
for s in a1('Some info about canada?'):
    print(s, end='')

 Absolutely, I'd be happy to share some information about Canada! Canada is a North American country situated primarily in the northern part of the continent. It is the second-largest country by land area and the third-largest by total area. Canada has a diverse population with both English and French as its official languages. It is known for its natural beauty with vast forests, mountains, lakes, and coastlines.

Canada gained its independence on July 1, 1867, through Confederation. It is a constitutional monarchy with Queen Elizabeth II as its monarch. The capital city of Canada is Ottawa, located in Ontario. The country has a diverse economy with sectors including agriculture, manufacturing, and services.

Canada is also known for its strong commitment to multiculturalism and social policies such as universal healthcare and education. It has a reputation for being peaceful and welcoming to immigrants and refugees. Some popular attractions include Niagara Falls, Banff National Park,

In [29]:
resp_to_json(
    a1('Some more info about canada?', response_format={'population': 'int', 'largest city': 'str of name'})
)

{'population': 37746023, 'largest city': 'Toronto'}

In [30]:
print(a1.get_hist())

system --- System Prompt: You are a deeply patriotic canadian assistant.
__________

user --- Some info about canada?
__________

assistant ---  Absolutely, I'd be happy to share some information about Canada! Canada is a North American country situated primarily in the northern part of the continent. It is the second-largest country by land area and the third-largest by total area. Canada has a diverse population with both English and French as its official languages. It is known for its natural beauty with vast forests, mountains, lakes, and coastlines.

Canada gained its independence on July 1, 1867, through Confederation. It is a constitutional monarchy with Queen Elizabeth II as its monarch. The capital city of Canada is Ottawa, located in Ontario. The country has a diverse economy with sectors including agriculture, manufacturing, and services.

Canada is also known for its strong commitment to multiculturalism and social policies such as universal healthcare and education. It ha